In [7]:
import numpy as np
import math as mt
import sympy as sp
import random as rd
import copy
import random

In [2]:
#미분법 1-1

# 변수 정의
x = sp.symbols('x')

# 함수 정의
f = 2*sp.sin(x) + 0.5*x

# 1차 미분
f_prime = sp.diff(f, x)

# 2차 미분
f_double_prime = sp.diff(f_prime, x)

# 1차 미분값이 0인 임계점 찾기
critical_points = sp.solveset(f_prime, x, domain=sp.Interval(-5, 13))

# 각 임계점에서 2차 미분 값 계산 (최솟값, 최댓값 판별)
second_derivative_at_critical = [f_double_prime.subs(x, pt) for pt in critical_points]

# 각 임계점에서 함수 값 계산
function_values_at_critical = [f.subs(x, pt) for pt in critical_points]

# 최소값을 찾기 위한 비교
min_value = min(function_values_at_critical)
min_index = function_values_at_critical.index(min_value)
min_x = critical_points.args[min_index]
min_y = min_value

# 출력
print(min_x, min_y)


-1.82347658193698 -2.84822996407220


In [3]:
#뉴톤법 1-2
# 변수 정의
x = sp.symbols('x')

# 함수 정의
f = 2*sp.sin(x) + 0.5*x

# 미분 정의
f_prime = sp.diff(f, x)

#뉴턴법 함수 정의
def newton_method(x_values):
    Differential_value = f_prime.subs(x, x_values).evalf()
    Function_value = f.subs(x, x_values).evalf()
    newton_method_value = x_values - Function_value/Differential_value
    return newton_method_value

# f(x) = 0값 찾기
num = 0
List_of_results = [13]  #초기값 설정
while True:
    if f.subs(x, List_of_results[-1]).evalf() == 0:
        print(f'x = {List_of_results[-1]} y = {f.subs(x, List_of_results[-1])}')
        break
    List_of_results.append(newton_method(List_of_results[num]))
    num += 1

x = 0 y = 0


In [4]:
# 경사 하강법 1-3

#변수 전의
x = sp.symbols('x')

#함수 정의
f = 2 * sp.sin(x) + 0.5 * x

# 미분해서 기울기(gradient) 구하기
f_prime = sp.diff(f, x)
f_prime_func = sp.lambdify(x, f_prime, modules=['numpy'])  # 수치 계산용 함수로 변환

# 초기값
x_val = 13

# 하이퍼파라미터
learning_rate = 1.7286
epochs = 1000

# 경사 하강법 수행
for epoch in range(epochs):
    grad = f_prime_func(x_val)
    x_val = x_val - learning_rate * grad 
    if round(grad, 1) == 0.0:   
        print(f"반복 횟수 {epoch}, x = {x_val:.6f}, 기울기 값 = {grad:.6f}")
        break


반복 횟수 9, x = 8.193441, 기울기 값 = -0.038631


In [5]:
#1-4 유전 알고리즘

# 목표 함수
def f(x):
    return 2*np.sin(x) + 0.5*x

# 미분 함수
def df(x):
    return 2*np.cos(x) + 0.5

# GA Parameters
POP_SIZE = 50
GENS = 100
MUT_RATE = 0.1
CROSS_RATE = 0.8
X_MIN, X_MAX = -5, 13
DERIV_TOL = 0.05  # 허용하는 미분값 오차

# 초기 집단
pop = np.random.uniform(low=X_MIN, high=X_MAX, size=POP_SIZE)

def fitness(x):
    deriv_penalty = np.abs(df(x))
    # 미분이 0에 가까울수록 좋고, f(x)는 작을수록 좋음
    return -f(x) - 10 * deriv_penalty  # 10배 패널티 가중치

for gen in range(GENS):
    # 적합도 평가
    fit_vals = fitness(pop)
    
    # 선택 (Roulette Wheel)
    probs = fit_vals - fit_vals.min() + 1e-6
    probs = probs / probs.sum()
    selected = np.random.choice(pop, size=POP_SIZE, p=probs)
    # 교차
    offspring = []
    for i in range(0, POP_SIZE, 2):
        if np.random.rand() < CROSS_RATE:
            alpha = np.random.rand()
            child1 = alpha*selected[i] + (1-alpha)*selected[i+1]
            child2 = alpha*selected[i+1] + (1-alpha)*selected[i]
            offspring.extend([child1, child2])
        else:
            offspring.extend([selected[i], selected[i+1]])
    
    # 변이
    for i in range(POP_SIZE):
        if np.random.rand() < MUT_RATE:
            offspring[i] += np.random.normal(scale=0.5)
            offspring[i] = np.clip(offspring[i], X_MIN, X_MAX)
    
    pop = np.array(offspring)

# 결과
best_idx = np.argmax(fitness(pop))
best_x = pop[best_idx]
print("최적 x:", best_x)
print(f"최소값: x:{best_x}, y:{f(best_x)}")



최적 x: -1.8207668039259177
최소값: x:-1.8207668039259177, y:-2.8482228526890903


In [10]:
#2번 

# 설정
GENE_LENGTH = 5  # x와 y 각각 5비트
CHROMOSOME_LENGTH = GENE_LENGTH * 2  # x + y
POPULATION_SIZE = 10
GENERATIONS = 30
MUTATION_RATE = 0.1
MUTATION_RATIO = 0.2  # 돌연변이 개체 비율

def decode(chromosome):
    x = int(chromosome[:GENE_LENGTH], 2)
    y = int(chromosome[GENE_LENGTH:], 2)
    return x, y

def is_feasible(x, y):
    return (
        -x + y <= 5 and
        x + y <= 10 and
        -2*x + y >= -10 and
        x >= 0 and y >= 0
    )

def fitness(chromosome):
    x, y = decode(chromosome)
    return x + 2*y if is_feasible(x, y) else 0

def initialize_population():
    population = []
    for _ in range(POPULATION_SIZE):
        x = random.randint(0, 31)
        y = random.randint(0, 31)
        chromosome = f"{x:05b}{y:05b}"
        population.append(chromosome)
    return population

def selection(population):
    # 적합도 기반 비례 선택 (룰렛 휠)
    total_fitness = sum(fitness(c) for c in population)
    if total_fitness == 0:
        return random.choices(population, k=POPULATION_SIZE)
    probabilities = [fitness(c)/total_fitness for c in population]
    return random.choices(population, weights=probabilities, k=POPULATION_SIZE)

def crossover(p1, p2):
    point = random.randint(1, CHROMOSOME_LENGTH - 1)
    c1 = p1[:point] + p2[point:]
    c2 = p2[:point] + p1[point:]
    return c1, c2

def mutate(chromosome):
    chromosome = list(chromosome)
    for i in range(len(chromosome)):
        if random.random() < MUTATION_RATE:
            chromosome[i] = '1' if chromosome[i] == '0' else '0'
    return ''.join(chromosome)

# 메인 알고리즘
population = initialize_population()

for generation in range(GENERATIONS):
    selected = selection(population)
    next_generation = []

    # 교차 (짝수 쌍으로 구성)
    for i in range(0, POPULATION_SIZE, 2):
        c1, c2 = crossover(selected[i], selected[i+1])
        next_generation.extend([c1, c2])

    # 돌연변이 적용 (20% 개체, 각 비트 10% 확률)
    num_mutate = int(MUTATION_RATIO * POPULATION_SIZE)
    indices = random.sample(range(POPULATION_SIZE), num_mutate)
    for idx in indices:
        next_generation[idx] = mutate(next_generation[idx])

    population = next_generation

# 결과 출력
best = max(population, key=lambda c: fitness(c))
x_best, y_best = decode(best)
print(f"최적 해: x = {x_best}, y = {y_best}, 목적함수값 = {fitness(best)}")

최적 해: x = 2, y = 6, 목적함수값 = 14


In [ ]:
#3번 문재

NUM_NODES = 8
POP_SIZE = 100
GENS = 200
MUT_RATE = 0.1

# 거리 행렬 정의
distance_matrix = np.array([
    [0, 2, 9, 10, 7, 3, 8, 6],
    [2, 0, 6, 4, 3, 5, 7, 9],
    [9, 6, 0, 8, 5, 4, 6, 3],
    [10, 4, 8, 0, 6, 2, 1, 5],
    [7, 3, 5, 6, 0, 3, 2, 4],
    [3, 5, 4, 2, 3, 0, 7, 8],
    [8, 7, 6, 1, 2, 7, 0, 3],
    [6, 9, 3, 5, 4, 8, 3, 0]
])

# 거리 계산 함수
def path_length(path):
    return sum(distance_matrix[path[i], path[i+1]] for i in range(NUM_NODES-1)) + distance_matrix[path[-1], path[0]]

# 초기 집단 생성
def init_population():
    return [random.sample(range(NUM_NODES), NUM_NODES) for _ in range(POP_SIZE)]

# 선택
def selection(population):
    return min(random.sample(population, 5), key=path_length)

# 교차 (Ordered Crossover)
def crossover(p1, p2):
    start, end = sorted(random.sample(range(NUM_NODES), 2))
    child = [None]*NUM_NODES
    child[start:end+1] = p1[start:end+1]
    fill = [n for n in p2 if n not in child]
    j = 0
    for i in range(NUM_NODES):
        if child[i] is None:
            child[i] = fill[j]
            j += 1
    return child

# 돌연변이 (Swap Mutation)
def mutate(path):
    if random.random() < MUT_RATE:
        i, j = random.sample(range(NUM_NODES), 2)
        path[i], path[j] = path[j], path[i]
    return path

# 메인 유전 알고리즘
def genetic_algorithm():
    population = init_population()
    for gen in range(GENS):
        new_pop = []
        for _ in range(POP_SIZE):
            p1, p2 = selection(population), selection(population)
            child = crossover(p1, p2)
            new_pop.append(mutate(child))
        population = new_pop
    best = min(population, key=path_length)
    return best, path_length(best)

# 실행
best_path, best_dist = genetic_algorithm()
print("최적 경로:", best_path)
print("최단 거리:", best_dist)
